In [ ]:
# 重构于notebooks文件夹下

import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 初始化模型参数

In [ ]:
# 本节将展平每个图像，把它们看作长度为784的向量。 在后面的章节中，我们将讨论能够利用图像空间结构的特征， 但现在我们暂时只把每个像素位置看作一个特征。
num_inputs = 784  # num_inputs 代表输入特征的数量。数据是从 28x28 的图像展平得到的。
num_outputs = 10  # num_outputs 代表输出的数量。对于 FashionMNIST 数据集，这通常是类别的数量，FashionMNIST 有 10 个类别，所以输出数量为 10。

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)  # 生成一个符合正态分布的张量，均值为 0，标准差为 0.01，形状为 [num_inputs, num_outputs]。这是为了初始化权重，防止初始权重全为零，导致梯度消失问题。
b = torch.zeros(num_outputs, requires_grad=True)  # 生成一个全零的张量，形状为 [num_outputs]。这用作偏置项的初始化。

# 权重 W 使用了正态分布初始化，偏置 b 初始化为零。这是一个常见的做法，用于线性分类模型或神经网络的线性层。

# 定义softmax操作

In [ ]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)
# X.sum(0, keepdim=True) 结果是一个形状为 [1, 3] 的张量，其中包含每列的和。
# X.sum(1, keepdim=True) 结果是一个形状为 [2, 1] 的张量，其中包含每行的和。

Softmax 函数是用于多类分类任务中将输出概率化的一种常用函数。它通常用于神经网络的输出层，以将网络的原始输出（即未归一化的得分）转化为概率分布。

### 数学公式

给定一个向量 $\mathbf{z} = [z_1, z_2, \ldots, z_K]$，其中 $K$ 是类别的数量，Softmax 函数的输出可以用以下公式表示：

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}
$$

#### 公式解释

- **$e^{z_i}$**：对于每个元素 $z_i$，我们首先计算它的指数函数 $e^{z_i}$。这是因为指数函数具有单调递增的性质，能够放大大值和缩小小值，从而使得结果更具区分度。
  
- **$\sum_{j=1}^K e^{z_j}$**：这是所有元素 $z_j$ 的指数函数之和，作为归一化因子。它确保所有输出值的总和为 1，从而使得每个输出值都可以解释为一个概率。

### 特性

1. **归一化**：
   Softmax 函数将输出转化为一个概率分布，所有输出值的总和为 1。

2. **非负性**：
   由于指数函数的输出始终是非负的，Softmax 函数的输出值也始终是非负的。

3. **分类概率**：
   每个输出值表示了输入属于该类别的概率，数值越高，表示该类别的可能性越大。

### 示例

假设有一个输入向量 $\mathbf{z} = [1.0, 2.0, 3.0]$，我们可以计算 Softmax 函数的输出如下：

1. **计算指数函数**：
   $$
   e^{z_1} = e^{1.0} \approx 2.718
   $$

   $$
   e^{z_2} = e^{2.0} \approx 7.389
   $$

   $$
   e^{z_3} = e^{3.0} \approx 20.086
   $$

2. **计算归一化因子**：
   $$
   \sum_{j=1}^3 e^{z_j} = e^{1.0} + e^{2.0} + e^{3.0} \approx 2.718 + 7.389 + 20.086 \approx 30.193
   $$

3. **计算 Softmax 输出**：
   $$
   \text{Softmax}(z_1) = \frac{e^{z_1}}{\sum_{j=1}^3 e^{z_j}} \approx \frac{2.718}{30.193} \approx 0.090
   $$
   $$
   \text{Softmax}(z_2) = \frac{e^{z_2}}{\sum_{j=1}^3 e^{z_j}} \approx \frac{7.389}{30.193} \approx 0.245
   $$
   $$
   \text{Softmax}(z_3) = \frac{e^{z_3}}{\sum_{j=1}^3 e^{z_j}} \approx \frac{20.086}{30.193} \approx 0.665
   $$

因此，Softmax 函数将输入向量 $\mathbf{z} = [1.0, 2.0, 3.0]$ 转换为概率分布 $\approx [0.090, 0.245, 0.665]$。

In [ ]:
# 用来计算一个二维张量 X 的 Softmax 函数的。
def softmax(X):
    X_exp = torch.exp(X)  # 计算张量 X 中每个元素的指数值。这个步骤将原始得分转换为正的指数值，准备进行归一化。
    partition = X_exp.sum(1, keepdim=True)  # 沿着第 1 维（即每行）计算指数值的和。计算得到的 partition 是一个形状为 [N, 1] 的张量，其中 N 是输入张量 X 的行数，表示每行的所有指数值之和。
    return X_exp / partition  # 归一化

In [ ]:
# 首先生成一个具有正态分布（均值为0，标准差为1）的随机张量 X，然后应用之前定义的 softmax 函数来计算 Softmax 概率分布，最后检查每行的概率总和是否为1。
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

# 定义模型

定义softmax操作后，我们可以实现softmax回归模型。 下面的代码定义了输入如何通过网络映射到输出。 注意，将数据传递到模型之前，我们使用reshape函数将每张原始图像展平为向量。

In [ ]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

# 定义损失函数

交叉熵损失函数是用于分类问题中的一种常见损失函数，尤其是在多类分类任务中。它衡量了预测概率分布与真实类别分布之间的差异。下面是对交叉熵损失函数的数学公式的详细讲解。

### 交叉熵损失函数的数学公式

#### 二分类交叉熵损失（Binary Cross-Entropy）

对于二分类问题，交叉熵损失函数的公式为：

$$
\text{Loss} = - \left( y \cdot \log(p) + (1 - y) \cdot \log(1 - p) \right)
$$

其中：
- \( y \) 是真实标签，取值为 0 或 1。
- \( p \) 是模型预测为正类的概率。

**解释：**
- 如果真实标签 \( y \) 为 1，则损失为 \( - \log(p) \)，这是对预测概率 \( p \) 的负对数。
- 如果真实标签 \( y \) 为 0，则损失为 \( - \log(1 - p) \)，这是对预测概率 \( 1 - p \) 的负对数。

#### 多分类交叉熵损失（Categorical Cross-Entropy）

对于多分类问题，交叉熵损失函数的公式为：

$$
\text{Loss} = - \sum_{i=1}^{K} y_i \cdot \log(p_i)
$$

其中：
- \( K \) 是类别的总数。
- \( y_i \) 是真实标签的 one-hot 编码向量中的第 \( i \) 个元素，如果样本的真实类别是第 \( i \) 类，则 \( y_i = 1 \)，否则 \( y_i = 0 \)。
- \( p_i \) 是模型对第 \( i \) 类的预测概率。

**解释：**
- 公式中只有真实类别的预测概率会影响损失，因为 \( y_i \) 只有在真实类别位置才是 1，其余位置为 0。
- 损失是所有类别中真实标签的预测概率的负对数之和。它衡量了模型预测的概率分布与真实分布之间的差距。

### 示例

假设我们有一个多分类问题，有 3 个类别，真实标签 \( y \) 是类别 1（即 \( y = [1, 0, 0] \)），模型的预测概率为 \( p = [0.1, 0.3, 0.6] \)。

**计算过程：**

1. 选择真实类别的预测概率：
   - 对于类别 1，真实标签的预测概率为 0.3。

2. 计算对数：
 $$\(\log(0.3) \approx -1.20397\)$$。

3. 计算交叉熵损失：
$$\(\text{Loss} = - \log(0.3) \approx 1.20397\)$$。

### 总结

交叉熵损失函数通过计算预测概率与真实标签之间的差异来评估模型的性能。它对模型的输出概率分布进行“惩罚”，如果预测概率离真实标签的概率较远，损失值会更大，从而引导模型进行更好的优化。

In [ ]:
y = torch.tensor([0, 2])  # 是一个包含真实类别索引的张量。
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])  # 是模型的预测概率分布，其中每行代表一个样本的类别概率。
y_hat[[0, 1], y]

In [ ]:
def cross_entropy(y_hat, y):
    # 定义交叉熵损失函数
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

# 分类精度

给定预测概率分布y_hat，当我们必须输出硬预测（hard prediction）时， 我们通常选择预测概率最高的类。 许多应用都要求我们做出选择。如Gmail必须将电子邮件分类为“Primary（主要邮件）”、 “Social（社交邮件）”“Updates（更新邮件）”或“Forums（论坛邮件）”。 Gmail做分类时可能在内部估计概率，但最终它必须在类中选择一个。

当预测与标签分类y一致时，即是正确的。 分类精度即正确预测数量与总预测数量之比。 虽然直接优化精度可能很困难（因为精度的计算不可导）， 但精度通常是我们最关心的性能衡量标准，我们在训练分类器时几乎总会关注它。

为了计算精度，我们执行以下操作。 首先，如果y_hat是矩阵，那么假定第二个维度存储每个类的预测分数。 我们使用argmax获得每行中最大元素的索引来获得预测类别。 然后我们将预测类别与真实y元素进行比较。 由于等式运算符“==”对数据类型很敏感， 因此我们将y_hat的数据类型转换为与y的数据类型一致。 结果是一个包含0（错）和1（对）的张量。 最后，我们求和会得到正确预测的数量。

In [ ]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    # 它首先将预测的概率分布转化为类别标签（如果需要的话），然后比较这些预测标签与真实标签，最终计算预测正确的数量。
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [ ]:
accuracy(y_hat, y) / len(y)

In [ ]:
# 定义了一个用于累加和评估模型准确性的工具。


class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
evaluate_accuracy(net, test_iter)

# 训练

在我们看过 3.2节中的线性回归实现， softmax回归的训练过程代码应该看起来非常眼熟。 在这里，我们重构训练过程的实现以使其可重复使用。 首先，我们定义一个函数来训练一个迭代周期。 请注意，updater是更新模型参数的常用函数，它接受批量大小作为参数。 它可以是d2l.sgd函数，也可以是框架的内置优化函数。

In [ ]:
def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

In [ ]:
# 在展示训练函数的实现之前，我们定义一个在动画中绘制数据的实用程序类Animator， 它能够简化本书其余部分的代码。

class Animator:  #@save
    """在动画中绘制数据"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [ ]:
# 接下来我们实现一个训练函数， 它会在train_iter访问到的训练数据集上训练一个模型net。 该训练函数将会运行多个迭代周期（由num_epochs指定）。 在每个迭代周期结束时，利用test_iter访问到的测试数据集对模型进行评估。 我们将利用Animator类来可视化训练进度。

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):  #@save
    """训练模型（定义见第3章）"""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
# 作为一个从零开始的实现，我们使用 3.2节中定义的 小批量随机梯度下降来优化模型的损失函数，设置学习率为0.1。
lr = 0.1

def updater(batch_size):
    return d2l.sgd([W, b], lr, batch_size)

In [ ]:
# 现在，我们训练模型10个迭代周期。 请注意，迭代周期（num_epochs）和学习率（lr）都是可调节的超参数。 通过更改它们的值，我们可以提高模型的分类精度。

num_epochs = 10
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)

# 预测

现在训练已经完成，我们的模型已经准备好对图像进行分类预测。 给定一系列图像，我们将比较它们的实际标签（文本输出的第一行）和模型预测（文本输出的第二行）。

In [ ]:
def predict_ch3(net, test_iter, n=6):  #@save
    """预测标签（定义见第3章）"""
    for X, y in test_iter:
        break
    trues = d2l.get_fashion_mnist_labels(y)
    preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true +'\n' + pred for true, pred in zip(trues, preds)]
    d2l.show_images(
        X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

predict_ch3(net, test_iter)